<a href="https://colab.research.google.com/github/maitreya-v/Colab/blob/main/%F0%9F%92%AE_GenerAds_%F0%9F%92%AEFine_tuning_BLOOM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune a BLOOM-based ad generation model using `peft`, `transformers` and `bitsandbytes`

We can use the [Product Descriptions and Ads Dataset](https://huggingface.co/datasets/c-s-ale/Product-Descriptions-and-Ads) to fine-tune BLOOM to be able to generate simple ads based off of product names, and descriptions! Perfect for Twitter or Instagram!

### Overview of PEFT and LoRA:

Based on some awesome new research [here](https://github.com/huggingface/peft), we can leverage techniques like PEFT and LoRA to train/fine-tune large models a lot more efficiently. 

It can't be explained much better than the overview given in the above link: 

```
Parameter-Efficient Fine-Tuning (PEFT) methods enable efficient adaptation of
pre-trained language models (PLMs) to various downstream applications without 
fine-tuning all the model's parameters. Fine-tuning large-scale PLMs is often 
prohibitively costly. In this regard, PEFT methods only fine-tune a small 
number of (extra) model parameters, thereby greatly decreasing the 
computational and storage costs. Recent State-of-the-Art PEFT techniques 
achieve performance comparable to that of full fine-tuning.
```

### Install requirements

First, run the cells below to install the requirements:

In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (p

### Model loading

Here let's load the `bloom-1b7` model!

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-1b7", 
    load_in_8bit=True, 
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/tokenizer")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


### Post-processing on the model

Finally, we need to apply some post-processing on the 8-bit model to enable training, let's freeze all our layers, and cast the layer-norm in `float32` for stability. We also cast the output of the last layer in `float32` for the same reasons.

In [3]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

### Apply LoRA

Here comes the magic with `peft`! Let's load a `PeftModel` and specify that we are going to use low-rank adapters (LoRA) using `get_peft_model` utility function from `peft`.

In [4]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [5]:
from peft import LoraConfig, get_peft_model 

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 3145728 || all params: 1725554688 || trainable%: 0.18230242262828822


### Preprocessing

We can simply load our dataset from 🤗 Hugging Face with the `load_dataset` method!

In [6]:
import transformers
from datasets import load_dataset

dataset_name = "c-s-ale/Product-Descriptions-and-Ads"
product_name = "product"
product_desc = "description"
product_ad = "ad"

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
dataset = load_dataset(dataset_name)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/90 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/c-s-ale___parquet/c-s-ale--Product-Descriptions-and-Ads-4c74ccc0a63de502/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
dataset['train'][:3]

{'product': [' Harem pants', ' Fringe skirt', ' Gingham dress'],
 'description': [' A style of pants with a dropped crotch, loose-fitting legs, and a gathered waistband for a unique, bohemian look.',
  ' A skirt featuring fringe detailing on the bottom, creating movement and fun.',
  ' A dress featuring a two-toned checkered pattern, often associated with picnics and summery outfits.'],
 'ad': ['Discover Harem Pants! Unique, stylish bohemian vibes with a dropped crotch & loose legs. Comfy meets chic - elevate your wardrobe. Limited stock - shop now!',
  'Introducing our fabulous Fringe Skirt! Step out in style with eye-catching fringe detailing that adds flair and movement. Perfect for any occasion, create unforgettable memories with this chic piece.',
  "Introducing the Gingham Dress: Timeless & Chic! 💕 Step into summer with this must-have, two-toned checkered dress. From picnics to parties, it's your go-to look. Shop now for unbeatable style!"]}

In [53]:
filename='/content/drive/MyDrive/Research Paper Dataset/maitreya.csv'

In [54]:
import pandas as pd

def dataframe_to_dict(df):
    # Create an empty dictionary to store the lists
    result_dict = {
        'repository': [],
        'pulls': [],
        'commits': [],
        'readme': [],
        'release_notes': []
    }
    
    # Iterate over the rows in the DataFrame and append the values to the corresponding lists
    for _, row in df.iterrows():
        result_dict['repository'].append(row['repository'])
        result_dict['pulls'].append(row['pulls'])
        result_dict['commits'].append(row['commits'])
        result_dict['readme'].append(row['readme'])
        result_dict['release_notes'].append(row['release_notes'])
    
    return result_dict

In [55]:
import pandas as pd
import numpy as np
dataset = pd.read_csv(filename)

# convert each row to a dictionary and store them in a list
data_list = []
for index, row in dataset.iterrows():
    data_list.append(row.to_dict())

# convert the list of dictionaries to a Pandas Series
dataset = pd.Series(data_list)

# print the first element of the series
print((dataset[0]))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



We want to put our data in the form:

```
Below is a product and description, please write an ad for this product.

### Product and Description:
PRODUCT NAME AND DESCRIPTION HERE

### Ad:
OUR AD HERE
```

This way, we can prompt our model well and receive the responses we want!

This is what fine-tuning, and prompt-engineering, is really all about!

In [56]:
# Define a new function to encode the prompt
def generate_prompt(repository: str, pulls: str, readme: str, commits: str, release_notes: str) -> str:
  prompt = f"Below is the repository name,pull request messages, commit messages and release notes.\n\n### Product and Description:\n{repository}: {pulls}: {commits}: {release_notes}\n\n### Readme:\n{readme}"
  return prompt

# def encode_prompt(prompt, max_length=512):
#   # print(encoded)
#   # encoded = tokenizer.encode(prompt, truncation=True, padding=True, max_length=max_length)
#   return {'input_ids': tokenizer.encode(prompt, truncation=True, padding=True, max_length=max_length), 'attention_mask': [1] * len(prompt)}
#   # return {'input_ids': encoded['input_ids'], 'attention_mask': encoded['attention_mask']}

def encode_prompt(prompt, max_length=512):
    encoded = tokenizer.encode_plus(prompt, truncation=True, padding=True, max_length=max_length)
    print(type(encoded))
    print(encoded)
    return {'input_ids': encoded['input_ids'], 'attention_mask': encoded['attention_mask']}



# Map the dataset using the new function
dataset = dataset.map(lambda samples: encode_prompt(generate_prompt(samples['repository'], samples['pulls'], samples['readme'], samples['commits'], samples['release_notes']), max_length=512))

# Train the model using the Trainer class
trainer = transformers.Trainer(
    model=model, 
    train_dataset=dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, 
        gradient_accumulation_steps=4,
        warmup_steps=100, 
        max_steps=100, 
        learning_rate=1e-3, 
        fp16=True,
        logging_steps=1, 
        output_dir='outputs',
        # max_length=512
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

<class 'transformers.tokenization_utils_base.BatchEncoding'>
{'input_ids': [111757, 632, 368, 55655, 4040, 15, 183280, 8821, 29866, 15, 29852, 29866, 530, 26963, 34383, 6149, 105311, 38518, 530, 224575, 189, 33613, 18, 4778, 17490, 29, 115379, 15921, 4105, 3262, 54072, 150487, 15701, 3331, 25932, 861, 10400, 664, 42015, 14039, 3727, 42015, 14039, 18, 69327, 139704, 4636, 1130, 132829, 6216, 4005, 632, 267, 17721, 78622, 613, 189255, 26996, 613, 97663, 427, 4005, 490, 4231, 18564, 427, 337, 2322, 530, 21907, 11435, 427, 41261, 8256, 64724, 18882, 23741, 361, 503, 2233, 361, 42015, 14039, 78622, 43219, 6997, 109517, 361, 3702, 127892, 44637, 85274, 124078, 17, 13292, 12657, 58226, 15270, 861, 219681, 147899, 3702, 127892, 71898, 104616, 3702, 127892, 77076, 40136, 19428, 17, 9757, 3262, 15980, 64362, 21723, 43834, 30347, 664, 15796, 18077, 14652, 30306, 18, 8240, 6013, 29733, 4073, 52674, 18077, 77076, 17721, 13512, 6222, 6281, 58246, 361, 229302, 530, 24985, 3359, 951, 3335, 181452, 388

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,2.670200
2,2.608000
3,2.605100
4,2.639300
5,2.636100
6,2.595100
7,2.492100
8,2.429900
9,2.478400
10,2.412600


TrainOutput(global_step=100, training_loss=0.9782705328590237, metrics={'train_runtime': 467.3796, 'train_samples_per_second': 3.423, 'train_steps_per_second': 0.214, 'total_flos': 1861251760128000.0, 'train_loss': 0.9782705328590237, 'epoch': 100.0})

In [57]:
# # Define a new function to encode the prompt
# def generate_prompt(repository: str, pulls: str, readme: str, commits: str, release_notes: str) -> str:
#   prompt = f"Below is the repository name,pull request messages, commit messages and release notes.\n\n### Pull request messages,Commit messages and Release notes:\n{repository}: {pulls}: {commits}: {release_notes}\n\n### Readme:\n{readme}"
#   return prompt

# def encode_prompt(prompt, max_length=512):
#   encoded = tokenizer.encode(prompt, truncation=True, padding=True, max_length=max_length)
#   return {'input_ids': encoded['input_ids'], 'attention_mask': encoded['attention_mask']}

# # Map the dataset using the new function
# dataset = dataset.map(lambda samples: encode_prompt(generate_prompt(samples['repository'], samples['pulls'], samples['readme'], samples['commits'], samples['release_notes']), max_length=512))

# # Train the model using the Trainer class
# trainer = transformers.Trainer(
#     model=model, 
#     train_dataset=dataset,
#     args=transformers.TrainingArguments(
#         per_device_train_batch_size=4, 
#         gradient_accumulation_steps=4,
#         warmup_steps=100, 
#         max_steps=100, 
#         learning_rate=1e-3, 
#         fp16=True,
#         logging_steps=1, 
#         output_dir='outputs',
#         max_length=512
#     ),
#     data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
# )
# model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
# trainer.train()


In [58]:
# def generate_prompt(repository: str, pulls: str, readme: str, commits: str, release_notes: str) -> str:
#   prompt = f"Below is the repository name,pull request messages, commit messages and release notes.\n\n### Product and Description:\n{repository}: {pulls}: {commits}: {release_notes}\n\n### Readme:\n{readme}"
#   return prompt

# dataset = dataset.map(lambda samples: tokenizer(generate_prompt(samples['repository'], samples['pulls'], samples['readme'], samples['commits'], samples['release_notes'])))

In [59]:
# trainer = transformers.Trainer(
#     model=model, 
#     train_dataset=dataset,
#     args=transformers.TrainingArguments(
#         per_device_train_batch_size=4, 
#         gradient_accumulation_steps=4,
#         warmup_steps=100, 
#         max_steps=100, 
#         learning_rate=1e-3, 
#         fp16=True,
#         logging_steps=1, 
#         output_dir='outputs'
#     ),
#     data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
# )
# model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
# trainer.train()

## Share adapters on the 🤗 Hub

Make sure you have a Hugging Face account, and you have set up a read/write token!

More info here: https://huggingface.co/docs/hub/security-tokens

In [60]:
HUGGING_FACE_USER_NAME = "MaitreyaV"

In [61]:
from huggingface_hub import notebook_login
notebook_login()

In [63]:
model.push_to_hub(f"{HUGGING_FACE_USER_NAME}/GenerAd-AI", use_auth_token=True)

CommitInfo(commit_url='https://huggingface.co/MaitreyaV/GenerAd-AI/commit/8bcaf4e092938ca9693731c2800902e2c1ca35f2', commit_message='Upload model', commit_description='', oid='8bcaf4e092938ca9693731c2800902e2c1ca35f2', pr_url=None, pr_revision=None, pr_num=None)

## Load adapters from the Hub

You can also directly load adapters from the Hub using the commands below:

In [64]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = f"{HUGGING_FACE_USER_NAME}/GenerAd-AI"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


## Inference

You can then directly use the trained model or the model that you have loaded from the 🤗 Hub for inference!

### Take it for a spin!

In [65]:
from IPython.display import display, Markdown

def make_inference(repository, pulls,commits,release_notes):
  batch = tokenizer(f"Below is the repository name,pull request messages, commit messages and release notes.\n\n### Product and Description:\n{repository}: {pulls}: {commits}: {release_notes}\n\n### Readme:", return_tensors='pt')

  with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=50)

  display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))

In [ ]:
your_repository_name_here = ""
your_repo_pulls_here = ""
# your_repo_commits_here = ""
# your_repo_release_notes_here = ""

# make_inference(your_repository_name_here, your_repo_pulls_here,your_repo_commits_here,your_repo_release_notes_here)
make_inference(your_repository_name_here, your_repo_pulls_here)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1405: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


### Product and Description:
Cool Shoes: Shoes with a racing stripe and stickers.

### Ad:
Discover cool shoes with a racing stripe and stickers! Get them now and stand out from the crowd in stunning style. Limited stock! Get yours now! #CoolShoes 🛠 RacingStripeStickers

### Example in Training Set

In [ ]:
batch = tokenizer("### Product and Description:\n Lace-up sandals: Shoes featuring laces or ties that wrap around the foot and, in some cases, the ankle.\n\n### Ad:", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 ### Product and Description:
 Lace-up sandals: Shoes featuring laces or ties that wrap around the foot and, in some cases, the ankle.

### Ad:
Discover timeless style with our Lace-up Sandals! Wrap your feet in chic design and experience unbeatable comfort. Perfect for any occasion, upgrade your shoe game today. Limited stock. Get yours now! Visit our Shoe


### Example outside of Training Set

In [ ]:
batch = tokenizer("### Product and Description:\nSundress: A flowery yellow sundress with blue polka dots. \n\n### Ad:", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 ### Product and Description:
 Sundress: A flowery yellow sundress with blue polka dots.

### Ad:
Discover timeless style in a flowery yellow sundress + blue polka dots! Make a statement in any occasion, featuring sheer perfection in every turn. Elevate your wardrobe now! #SundressLove #F


### Example outside of immediate domain

In [ ]:
batch = tokenizer("### Product and Description:\n A new Lexus: A luxury automobile with grey paint and tinted windows.\n\n### Ad:", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 ### Product and Description:
 A new Lexus: A luxury automobile with grey paint and tinted windows.

### Ad:
Discover luxury and style with our new Lexus! Experience grey paint & tinted windows, two of the most sought-after luxury attributes. Discover your new calling today. #LexusLuxury #LexusTintedWindows


As you can see by fine-tuning for few steps we have almost recovered the exact quote from the training data.